In [6]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# Carregar os dados
tcc_data = pd.read_csv("dadosTCC_final.csv")

# Preparar variáveis dependentes e independentes
dependent_var = tcc_data['Valor Desembolsado em Reais']
df_dummy = tcc_data.drop(columns=['UF','Ano', 'Cliente', 'Valor Desembolsado em Reais', 'Subsetor CNAE Agrupado', 'Instituição Financeira Credenciada', 'Valor da Operação em R$'])

# df_dummy.drop(df_dummy.tail(1).index,inplace=True) # drop last n rows

# Criar variáveis dummy para dados categóricos
df_dummy = pd.get_dummies(df_dummy, drop_first=True)

# Remover qualquer linha com NaN e converter para tipo numérico
df_dummy = df_dummy.dropna().astype(float)  # Assegura que todos os dados são float

# Adicionar coluna constante para intercepto
df_dummy = sm.add_constant(df_dummy)  # Adiciona uma coluna constante para o intercepto

# Garantir que dependent_var está alinhada com df_dummy
tcc_data = tcc_data.loc[df_dummy.index]  # Alinha tcc_data com df_dummy
dependent_var = tcc_data['Valor Desembolsado em Reais']

# Resetar índices para garantir alinhamento
dependent_var = dependent_var.reset_index(drop=True)
df_dummy = df_dummy.reset_index(drop=True)

In [7]:
df_dummy.columns

Index(['const', 'Contribuição do UF ao PIB Nacional (%)',
       'Proporção do Valor Setorial ao PIB do UF',
       'Setor CNAE_COMERCIO E SERVICOS',
       'Setor CNAE_INDUSTRIA DE TRANSFORMAÇÃO',
       'Setor CNAE_INDUSTRIA EXTRATIVA', 'Porte do cliente_MICRO',
       'Porte do cliente_MÉDIA', 'Porte do cliente_PEQUENA'],
      dtype='object')

In [8]:
# Criar o modelo de regressão OLS
model = sm.OLS(dependent_var, df_dummy)

# Ajustar o modelo
results = model.fit()

# Salvando os resultados em um csv
tables = results.summary().tables
dataframes = [pd.DataFrame(table[1:], columns=table[0]) for table in tables]

csv_path = 'regTCC_sem_contribuicao_com_UF_.csv'

with open(csv_path, 'w') as f:
    for i, df in enumerate(dataframes):
        f.write(f"\nTable {i + 1}: {results.summary().extra_txt[i].strip()}\n")
        
        df.to_csv(f, index=False)

        f.write('\n')
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Valor Desembolsado em Reais   R-squared:                       0.035
Model:                                     OLS   Adj. R-squared:                  0.034
Method:                          Least Squares   F-statistic:                     32.43
Date:                         Wed, 29 May 2024   Prob (F-statistic):           1.56e-50
Time:                                 13:39:27   Log-Likelihood:            -1.3458e+05
No. Observations:                         7079   AIC:                         2.692e+05
Df Residuals:                             7070   BIC:                         2.692e+05
Df Model:                                    8                                         
Covariance Type:                     nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
const                                     1.035e+07   3.25e+06      3.185      0.001    3.98e+06    1.67e+07
Contribuição do UF ao PIB Nacional (%)     3.01e+07   4.54e+06      6.637      0.000    2.12e+07     3.9e+07
Proporção do Valor Setorial ao PIB do UF  1.228e+06   1.88e+06      0.653      0.513   -2.46e+06    4.91e+06
Setor CNAE_COMERCIO E SERVICOS            8.582e+05   4.23e+06      0.203      0.839   -7.43e+06    9.15e+06
Setor CNAE_INDUSTRIA DE TRANSFORMAÇÃO     4.549e+06    3.3e+06      1.377      0.168   -1.93e+06     1.1e+07
Setor CNAE_INDUSTRIA EXTRATIVA           -1.111e+06    1.3e+07     -0.085      0.932   -2.66e+07    2.44e+07
Porte do cliente_MICRO                   -1.746e+07   6.35e+06     -2.752      0.006   -2.99e+07   -5.02e+06
Porte do cliente_MÉDIA                   -1.736e+07   1.41e+06    -12.273      0.000   -2.01e+07   -1.46e+07
Porte do cliente_PEQUENA                 -1.674e+07    4.6e+06     -3.637      0.000   -2.58e+07   -7.72e+06
==============================================================================
Omnibus:                    16247.828   Durbin-Watson:                   1.729
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        205114446.097
Skew:                          21.879   Prob(JB):                         0.00
Kurtosis:                     835.758   Cond. No.                         35.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""